# 백테스팅 사례 연구: 변동성 돌파 전략

이 노트북은 crypto-lab을 사용하여 변동성 돌파(Volatility Breakout) 전략을 백테스팅하는 과정을 보여줍니다.

## 1. 환경 설정 및 필수 라이브러리

필요한 라이브러리를 import하고 설정을 초기화합니다.

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

# Import from crypto-lab
from src.backtester.engine import BacktestEngine
from src.config.loader import load_config
from src.data.collector_factory import CollectorFactory
from src.strategies.volatility_breakout import VanillaVBO

# Set up visualization
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

print("✓ 모든 라이브러리가 성공적으로 로드되었습니다.")

## 2. 데이터 수집 및 준비

Upbit 거래소에서 BTC/KRW 데이터를 수집합니다.

In [ ]:
# Load configuration
config = load_config()

# Collect data
collector = CollectorFactory.create("upbit", "BTC/KRW")
df = collector.collect(days=365)

# Display basic information
print(f"Data Shape: {df.shape}")
print(f"Date Range: {df.index[0]} to {df.index[-1]}")
print("\nFirst few rows:")
print(df.head())
print("\nData Info:")
print(df.info())

## 3. 전략 설정 및 백테스팅 실행

변동성 돌파 전략을 설정하고 백테스팅을 실행합니다.

In [ ]:
# Create strategy
strategy = VanillaVBO(
    name="Volatility Breakout Strategy", lookback_period=20, atr_period=10, volatility_threshold=1.0
)

# Initialize backtester
backtester = BacktestEngine(
    initial_balance=10_000_000,  # 1000만 원
    trading_fees=0.0005,  # 0.05%
    slippage=0.001,  # 0.1%
)

# Run backtest
results = backtester.run(df, strategy)

print("✓ 백테스팅이 완료되었습니다.")
print("\n성과 요약:")
print(f"- 총 거래 수: {results['total_trades']}")
print(f"- 승리 거래: {results['winning_trades']}")
print(f"- 최종 수익: {results['total_return']:.2%}")
print(f"- 샤프 비율: {results['sharpe_ratio']:.2f}")

## 4. 성과 지표 분석

전략의 성과를 다양한 지표로 분석합니다.

In [ ]:
# Create metrics summary
metrics = {
    "총 수익률": f"{results['total_return']:.2%}",
    "연율화 수익률 (CAGR)": f"{results['cagr']:.2%}",
    "샤프 비율": f"{results['sharpe_ratio']:.2f}",
    "소르티노 비율": f"{results['sortino_ratio']:.2f}",
    "칼마 비율": f"{results['calmar_ratio']:.2f}",
    "최대 낙폭": f"{results['max_drawdown']:.2%}",
    "승률": f"{results['win_rate']:.2%}",
    "평균 거래 수익": f"{results['avg_trade_return']:.2%}",
}

metrics_df = pd.DataFrame(metrics.items(), columns=["지표", "값"])
print(metrics_df.to_string(index=False))

## 5. 자산 곡선 시각화

시간에 따른 포트폴리오 가치의 변화를 시각화합니다.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Plot equity curve
ax.plot(results["dates"], results["equity"], linewidth=2, label="포트폴리오 가치")
ax.fill_between(results["dates"], results["equity"], alpha=0.3)

ax.set_xlabel("날짜")
ax.set_ylabel("포트폴리오 가치 (원)")
ax.set_title("자산 곡선: 변동성 돌파 전략")
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("✓ 자산 곡선 표시 완료")

## 6. 월별 수익률 히트맵

월별/연도별 수익률을 히트맵으로 시각화합니다.

In [ ]:
# Create monthly returns heatmap
monthly_returns = results["monthly_returns"]

fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(
    monthly_returns,
    annot=True,
    fmt=".1%",
    cmap="RdYlGn",
    center=0,
    cbar_kws={"label": "수익률"},
    ax=ax,
)

ax.set_title("월별 수익률 히트맵")
ax.set_xlabel("월")
ax.set_ylabel("연도")
plt.tight_layout()
plt.show()

print("✓ 월별 수익률 히트맵 표시 완료")

## 7. 거래 분석

개별 거래의 수익/손실을 분석합니다.

In [ ]:
# Analyze trades
trades = results["trades"]
trade_returns = [t["return"] for t in trades]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Profit Distribution
axes[0, 0].hist(trade_returns, bins=30, edgecolor="black", alpha=0.7)
axes[0, 0].set_title("거래 수익 분포")
axes[0, 0].set_xlabel("수익률")
axes[0, 0].set_ylabel("빈도")

# 2. Cumulative P&L
cum_pnl = np.cumsum([t["pnl"] for t in trades])
axes[0, 1].plot(cum_pnl, linewidth=2)
axes[0, 1].set_title("누적 손익")
axes[0, 1].set_xlabel("거래 번호")
axes[0, 1].set_ylabel("손익 (원)")
axes[0, 1].grid(True, alpha=0.3)

# 3. Win/Loss Distribution
win_losses = pd.Series(trade_returns).apply(lambda x: "Win" if x > 0 else "Loss")
win_losses.value_counts().plot(kind="bar", ax=axes[1, 0], color=["green", "red"])
axes[1, 0].set_title("승리/손실 거래 수")
axes[1, 0].set_ylabel("거래 수")

# 4. Drawdown
equity = results["equity"]
running_max = np.maximum.accumulate(equity)
drawdown = (equity - running_max) / running_max
axes[1, 1].fill_between(results["dates"], drawdown, alpha=0.5, color="red")
axes[1, 1].set_title("낙폭")
axes[1, 1].set_ylabel("낙폭")
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ 거래 분석 완료")

## 8. 결론 및 개선 사항

### 주요 발견사항:

1. **성과**: 변동성 돌파 전략은 일정한 수준의 수익성을 보여주었습니다.
2. **위험**: 최대 낙폭은 관리 가능한 수준으로 유지되었습니다.
3. **효율성**: 샤프 비율은 전략의 위험-조정 수익성을 나타냅니다.

### 개선 사항:

1. **파라미터 최적화**: 룩백 기간과 변동성 임계값을 최적화합니다.
2. **리스크 관리**: Stop-loss와 position sizing을 추가합니다.
3. **필터링**: 시장 변동성이 낮은 기간을 필터링합니다.
4. **다중 자산**: 다양한 암호화폐에 적용합니다.

### 실제 운영 전 체크리스트:

- [ ] Out-of-sample 테스트 완료
- [ ] Walk-forward 분석 수행
- [ ] Paper trading으로 검증
- [ ] 거래소 API 연동 테스트
- [ ] 리스크 관리 규칙 설정
- [ ] 모니터링 시스템 구축